In [88]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from configja import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [89]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/Rica6/Desktop/Python/APIs/WeatherPy_Database.csv")
city_data_df.head()

Unnamed: 0         City Country     Lat     Lng  Max Temp  Humidity  \
0           0  Port-Gentil      GA -0.7193  8.7815     78.76        83   
1           1         Axim      GH  4.8699 -2.2405     75.58        90   
2           2   Cape Coast      GH  5.1053 -1.2466     78.33        84   
3           3       Harper      LR  4.3750 -7.7169     75.27        93   
4           4     Abonnema      NG  4.7084  6.7931     75.06        90   

   Cloudiness  Wind Speed Current Description  
0          20        6.91          few clouds  
1         100        6.96     overcast clouds  
2         100        7.34          light rain  
3         100        8.79       moderate rain  
4          97        1.86     overcast clouds

In [93]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# User Selection for temperature values
user_min_temp = float(input("Please enter minimum temperature you would like for your trip? "))
user_max_temp = float(input("Please enter maximum temperature you would like for your trip? "))

Please enter minimum temperature you would like for your trip? 75
Please enter maximum temperature you would like for your trip? 98


In [94]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= user_max_temp) & \
                                       (city_data_df["Max Temp"] >= user_min_temp)]
preferred_cities_df.head(10)

Unnamed: 0         City Country     Lat     Lng  Max Temp  Humidity  \
0            0  Port-Gentil      GA -0.7193  8.7815     78.76        83   
1            1         Axim      GH  4.8699 -2.2405     75.58        90   
2            2   Cape Coast      GH  5.1053 -1.2466     78.33        84   
3            3       Harper      LR  4.3750 -7.7169     75.27        93   
4            4     Abonnema      NG  4.7084  6.7931     75.06        90   
6            6        Mbini      GQ  1.5740  9.6134     76.86        87   
8            8         Luba      GQ  3.4568  8.5547     78.69        84   
10          10     Takoradi      GH  4.8845 -1.7554     76.96        86   
13          13        Limbe      CM  4.0242  9.2149     78.04        84   
14          14      Mumford      GH  5.2618 -0.7590     78.87        83   

    Cloudiness  Wind Speed Current Description  
0           20        6.91          few clouds  
1          100        6.96     overcast clouds  
2          100        7.34          light rain  
3          100        8.79       moderate rain  
4           97        1.86     overcast clouds  
6          100        8.88     overcast clouds  
8          100        5.97     overcast clouds  
10          96        7.56     overcast clouds  
13         100        4.72     overcast clouds  
14          74        7.58       broken clouds

In [95]:
 # 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
preferred_cities_df.count()

Unnamed: 0             34
City                   34
Country                34
Lat                    34
Lng                    34
Max Temp               34
Humidity               34
Cloudiness             34
Wind Speed             34
Current Description    34
dtype: int64

In [96]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [97]:
 # 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

City Country  Max Temp Current Description     Lat     Lng  \
0   Port-Gentil      GA     78.76          few clouds -0.7193  8.7815   
1          Axim      GH     75.58     overcast clouds  4.8699 -2.2405   
2    Cape Coast      GH     78.33          light rain  5.1053 -1.2466   
3        Harper      LR     75.27       moderate rain  4.3750 -7.7169   
4      Abonnema      NG     75.06     overcast clouds  4.7084  6.7931   
6         Mbini      GQ     76.86     overcast clouds  1.5740  9.6134   
8          Luba      GQ     78.69     overcast clouds  3.4568  8.5547   
10     Takoradi      GH     76.96     overcast clouds  4.8845 -1.7554   
13        Limbe      CM     78.04     overcast clouds  4.0242  9.2149   
14      Mumford      GH     78.87       broken clouds  5.2618 -0.7590   

   Hotel Name  
0              
1              
2              
3              
4              
6              
8              
10             
13             
14

In [98]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [99]:
 hotel_df.head(10)

City Country  Max Temp Current Description     Lat     Lng  \
0   Port-Gentil      GA     78.76          few clouds -0.7193  8.7815   
1          Axim      GH     75.58     overcast clouds  4.8699 -2.2405   
2    Cape Coast      GH     78.33          light rain  5.1053 -1.2466   
3        Harper      LR     75.27       moderate rain  4.3750 -7.7169   
4      Abonnema      NG     75.06     overcast clouds  4.7084  6.7931   
6         Mbini      GQ     76.86     overcast clouds  1.5740  9.6134   
8          Luba      GQ     78.69     overcast clouds  3.4568  8.5547   
10     Takoradi      GH     76.96     overcast clouds  4.8845 -1.7554   
13        Limbe      CM     78.04     overcast clouds  4.0242  9.2149   
14      Mumford      GH     78.87       broken clouds  5.2618 -0.7590   

                     Hotel Name  
0            Fengshui Residence  
1     AXIM BEACH RESORT AND SPA  
2                  Samrit Hotel  
3                   Screensaver  
4        Corper's Lodge Obonoma  
6                                
8                                
10   Raybow International Hotel  
13  The LK Hotel at Bonadikombo  
14                Spanish Villa

In [100]:
 # 7. Drop the rows where there is no Hotel Name.
print(hotel_df.isnull().sum())
print(hotel_df.notnull().sum())

clean_hotel_df =hotel_df.dropna("Hotel Name" == "")
clean_hotel_df =hotel_df.dropna(how='any')

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64
City                   34
Country                34
Max Temp               34
Current Description    34
Lat                    34
Lng                    34
Hotel Name             34
dtype: int64


In [101]:
clean_hotel_df.head(10)

City Country  Max Temp Current Description     Lat     Lng  \
0   Port-Gentil      GA     78.76          few clouds -0.7193  8.7815   
1          Axim      GH     75.58     overcast clouds  4.8699 -2.2405   
2    Cape Coast      GH     78.33          light rain  5.1053 -1.2466   
3        Harper      LR     75.27       moderate rain  4.3750 -7.7169   
4      Abonnema      NG     75.06     overcast clouds  4.7084  6.7931   
6         Mbini      GQ     76.86     overcast clouds  1.5740  9.6134   
8          Luba      GQ     78.69     overcast clouds  3.4568  8.5547   
10     Takoradi      GH     76.96     overcast clouds  4.8845 -1.7554   
13        Limbe      CM     78.04     overcast clouds  4.0242  9.2149   
14      Mumford      GH     78.87       broken clouds  5.2618 -0.7590   

                     Hotel Name  
0            Fengshui Residence  
1     AXIM BEACH RESORT AND SPA  
2                  Samrit Hotel  
3                   Screensaver  
4        Corper's Lodge Obonoma  
6                                
8                                
10   Raybow International Hotel  
13  The LK Hotel at Bonadikombo  
14                Spanish Villa

In [102]:
# 8a. Create the output File (CSV)
output_data_file = 'C:/Users/Rica6/Desktop/Python/APIs/Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [103]:
 # 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [104]:
 # 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))